In [1]:
# add resources/utils to the python path
import sys
utils_path = os.path.abspath(os.path.join('..', 'resources', 'utils'))
if utils_path not in sys.path:
    sys.path.append(utils_path)

# util imports
from funman_demo.sim.CHIME.CHIME_SIR import main as run_CHIME_SIR
from funman_demo.sim.CHIME.CHIME_SIR_sir_function import main as run_CHIME_SIR_BL

# standard imports
from funman.parameter_space import ParameterSpace
from funman.model2smtlib.gromet.translate import QueryableGromet, QueryableBilayer

from pysmt.shortcuts import get_model, And, Symbol, FunctionType, Function, Equals, Int, Real, substitute, TRUE, FALSE, Iff, Plus, ForAll, LT, simplify
from pysmt.typing import INT, REAL, BOOL

import unittest
import os

In [4]:
RESOURCES = "../resources"
GROMET_FILE_1 = os.path.join(RESOURCES, "gromet", "CHIME_SIR_while_loop--Gromet-FN-auto.json")
GROMET_FILE_2 = os.path.join(RESOURCES, "gromet", "CHIME_SIR_while_loop--Gromet-FN-auto-one-epoch.json")

In [5]:
"""
This test constructs two formulations of the CHIME model:

   - the original model where Beta is a epoch-dependent constant over three
   epochs (i.e., a triple of parameters)

   - a modified variant of the original model using a single constant Beta over
   the entire simulation (akin to a single epoch).

It then compares the models by determining that the respective spaces of
feasible parameter values intersect.
"""
############################ Prepare Models ####################################
# read in the gromet files
# GROMET_FILE_1 is the original GroMEt extracted from the simulator
# It sets N_p = 3 and n_days = 20, resulting in three epochs of 0, 20, and 20 days
gromet_org = QueryableGromet.from_gromet_file(GROMET_FILE_1)
# GROMET_FILE_2 modifes sets N_p = 2 and n_days = 40, resulting in one epoch of 40 days
gromet_sub = QueryableGromet.from_gromet_file(GROMET_FILE_2)
# Scenario query threshold
infected_threshold = 130
############################ Evaluate Models ###################################
# get parameter space for the original (3 epochs)
ps_b1_b2_b3 = gromet_org.synthesize_parameters(f"(forall ((t Int)) (<= (I t) {infected_threshold}))")
# get parameter space for the constant beta variant
ps_bc = gromet_sub.synthesize_parameters(f"(forall ((t Int)) (<= (I t) {infected_threshold}))")

############################ Compare Models ####################################
# construct special parameter space where parameters are equal
ps_eq = ParameterSpace.construct_all_equal(ps_b1_b2_b3)
# intersect the original parameter space with the ps_eq to get the
# valid parameter space where (b1 == b2 == b3)
ps = ParameterSpace.intersect(ps_b1_b2_b3, ps_eq)
# assert the parameters spaces for the original and the constant beta
# variant are the same
#assert(ParameterSpace.compare(ps_bc, ps))

AttributeError: 'QueryableGromet' object has no attribute 'synthesize_parameters'

In [6]:
def compare_against_CHIME_FN(gromet_path, infected_threshold):
    """
    This function compares the simulator and FUNMAN reasoning about the CHIME
    SIR model.  The query_simulator function executes the simulator main() as
    run_CHIME_SIR, and answers the does_not_cross_threshold() query using the
    simulation results.  The QueryableGromet class constructs a model from the
    GroMEt file corresponding to the simulator, and answers a query with the
    model.  The query for both cases asks whether the number of infected at any
    time point exceed a specified threshold.  The test will succeed if the
    simulator and QueryableGromet class agree on the response to the query.  
    
    Args:
        infected_threshold (int): The upper bound for the number of infected for any time point.      
    Returns:
        bool: Do the simulator and QueryableGromet results match?
    """
    # query the simulator
    def does_not_cross_threshold(sim_results):
        i = sim_results[2]
        return all(i_t <= infected_threshold for i_t in i)
    q_sim = does_not_cross_threshold(run_CHIME_SIR())
    # query the gromet file
    gromet = QueryableGromet.from_gromet_file(gromet_path)
    q_gromet = gromet.query(f"(forall ((t Int)) (<= (I t) {infected_threshold}))")
    # assert the both queries returned the same result
    return  q_sim == q_gromet

In [7]:
"""

"""
infected_threshold = 130
compare_against_CHIME_FN(GROMET_FILE_1, infected_threshold)

s_n: 806.1042478004072
i_n: 120.8076619511338
r_n: 75.088090248459
E: [2, 2.067857142857143, 2.135467350628644, 2.2028312269522377, 2.2699493765097256, 2.336822405000486, 2.403450919115151, 2.4698355265094607, 2.5359768357782975, 2.6018754564298945, 2.667531998860216, 2.732947074327523, 2.7981212949271037, 2.8630552735661876, 2.9277496239390333, 2.9922049605021908, 3.0564218984499396, 3.1204010536899034, 3.1841430428188353, 3.2476484830985877, 3.3109179924322456, 3.5051753287837535, 3.7260841164428546, 3.977285156398101, 4.262912006758591, 4.587656321669678, 4.95684145580987, 5.3765052615712765, 5.853493073608701, 6.395561935605904, 7.011497171349971, 7.711242429149318, 8.506044325529986, 9.40861276828314, 10.433297933929914, 11.59628468965819, 12.915804958539258, 14.41236809679874, 16.109008743291326, 18.0315507662581, 20.20888481455726, 22.673255515119525, 25.460552473490456, 28.610596853765703, 32.167412362753566, 36.17946587530282, 40.69985867115213, 45.78644430904047, 51.501843611

True

In [8]:
"""

"""
infected_threshold = 100
compare_against_CHIME_FN(GROMET_FILE_1, infected_threshold)

s_n: 806.1042478004072
i_n: 120.8076619511338
r_n: 75.088090248459
E: [2, 2.067857142857143, 2.135467350628644, 2.2028312269522377, 2.2699493765097256, 2.336822405000486, 2.403450919115151, 2.4698355265094607, 2.5359768357782975, 2.6018754564298945, 2.667531998860216, 2.732947074327523, 2.7981212949271037, 2.8630552735661876, 2.9277496239390333, 2.9922049605021908, 3.0564218984499396, 3.1204010536899034, 3.1841430428188353, 3.2476484830985877, 3.3109179924322456, 3.5051753287837535, 3.7260841164428546, 3.977285156398101, 4.262912006758591, 4.587656321669678, 4.95684145580987, 5.3765052615712765, 5.853493073608701, 6.395561935605904, 7.011497171349971, 7.711242429149318, 8.506044325529986, 9.40861276828314, 10.433297933929914, 11.59628468965819, 12.915804958539258, 14.41236809679874, 16.109008743291326, 18.0315507662581, 20.20888481455726, 22.673255515119525, 25.460552473490456, 28.610596853765703, 32.167412362753566, 36.17946587530282, 40.69985867115213, 45.78644430904047, 51.501843611

False

In [9]:
def compare_against_CHIME_bilayer(infected_threshold):
    """
    This function compares the simulator and FUNMAN reasoning about the CHIME
    SIR model.  The query_simulator function executes the simulator main() as
    run_CHIME_SIR, and answers the does_not_cross_threshold() query using the
    simulation results.  The QueryableBilayer class constructs a model from the
    Bilayer file corresponding to the simulator, and answers a query with the
    model.  The query for both cases asks whether the number of infected exceed
    a specified threshold.  The test will succeed if the simulator and
    QueryableBilayer class agree on the response to the query.  
    
    Args:
        infected_threshold (int): The upper bound for the number of infected.      
    Returns:
        bool: Do the simulator and QueryableGromet results match?
    """
    # query the simulator
    def does_not_cross_threshold(sim_results):
        i = sim_results[1]
        return (i <= 5)
    q_sim = does_not_cross_threshold(run_CHIME_SIR_BL())
    print("q_sim", q_sim)
    # query the gromet file
    q_bilayer = QueryableBilayer.query(f"(i <= 5)")
    print("q_bilayer", q_bilayer)
    # assert the both queries returned the same result
    return  q_sim == q_bilayer

In [10]:
"""

"""
infected_threshold = 100
compare_against_CHIME_bilayer(infected_threshold)

(s,i,r)= (90.0, 10.7, 0.3)
q_sim False
q_bilayer False


True

In [11]:
"""
Encoding for `x = 2`
"""
gFile = os.path.join(RESOURCES, "gromet", "exp0--Gromet-FN-auto.json")


fn = QueryableGromet.from_gromet_file(gFile)
#print(fn._gromet_fn)
phi = fn.to_smtlib()
print("PHI:\n", phi)
model = get_model(phi)
assert(model) # Is phi satisfiable?
print("MODEL:\n", model)
assert(model.get_py_value(Symbol('exp0.fn.x', INT)) == 2) # Did the model get the right assignment?

PHI:
 (('exp0.fn.pof[0]' = 'exp0.fn.bf[0].value.fn.opo[0]') & (exp0.fn.x = 'exp0.fn.pof[0]') & ((('exp0.fn.bf[0].value.fn.pof[0]' = 'exp0.fn.bf[0].value.fn.bf[0].value') & ('exp0.fn.bf[0].value.fn.bf[0].value' = 2)) & ('exp0.fn.bf[0].value.fn.opo[0]' = 'exp0.fn.bf[0].value.fn.pof[0]')))
MODEL:
 'exp0.fn.pof[0]' := 2
'exp0.fn.bf[0].value.fn.opo[0]' := 2
'exp0.fn.bf[0].value.fn.bf[0].value' := 2
'exp0.fn.bf[0].value.fn.pof[0]' := 2
exp0.fn.x := 2
